In [227]:
# So we can load files from other sub-directories, e.g. datasets.
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path = [module_path] + sys.path
module_path = os.path.abspath(os.path.join('../unlabeled_extrapolation'))
if module_path not in sys.path:
    sys.path = [module_path] + sys.path

import torch
from torch import nn
import torchvision.transforms as transforms
import torchvision.models as models

import json
import numpy as np
from scipy.special import softmax
from sklearn.linear_model import LogisticRegression, SGDClassifier
import quinine
from unlabeled_extrapolation.baseline_train import preprocess_config, get_all_test_stats, get_test_loaders, build_model
from unlabeled_extrapolation.log_reg_sk import inv_normalize_weights
import unlabeled_extrapolation.utils.utils as utils
import unlabeled_extrapolation.datasets.breeds
from unlabeled_extrapolation.models import set_linear_layer

import calibration as cal
import unlabeled_extrapolation
import unlabeled_extrapolation.models
import pickle

import importlib
importlib.reload(imnet_resnet)
importlib.reload(unlabeled_extrapolation.models)

<module 'unlabeled_extrapolation.models' from '/juice/scr/ananya/cifar_experiments/unlabeled_extrapolation/unlabeled_extrapolation/models/__init__.py'>

In [185]:
# Get sklearn weights.
path = '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/logs/linprobe_living17_resnet50/weights_0.pkl'
coef, intercept, best_c, best_i = pickle.load( open( path, "rb" ) )
# TODO: important, this is not needed long term so comment this line and uncomment the next line.
# Only used as a stop-grab because we didn't apply this before saving.
# new_coef, new_intercept = inv_normalize_weights(coef, intercept, features, 0)
new_coef, new_intercept = coef, intercept

In [186]:
# Get moco model, and initialize datasets.
config_path = '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/configs/adaptation/living17.yaml'
config = quinine.Quinfig(config_path)
preprocess_config(config, config_path)
config.train_dataset.classname = 'unlabeled_extrapolation.' + config.train_dataset.classname
for test_dataset_config in config.test_datasets:
    test_dataset_config.classname = 'unlabeled_extrapolation.' + test_dataset_config.classname
config.model.args.checkpoint_path = '/u/scr/ananya/simclr_weights/moco_v2_800ep_pretrain.pth.tar'
net = build_model(config)
net.cuda()
test_loaders, max_examples = get_test_loaders(config, shuffle=False)

In [233]:
# Insert sklearn model into pytorch.
unlabeled_extrapolation.models.set_linear_layer(net._model.fc, new_coef, new_intercept)
# coef_tensor = torch.tensor(new_coef, dtype=net._model.fc.weight.dtype).cuda()
# bias_tensor = torch.tensor(new_intercept, dtype=net._model.fc.bias.dtype).cuda()
# coef_param = torch.nn.parameter.Parameter(coef_tensor)
# bias_param = torch.nn.parameter.Parameter(bias_tensor)
# net._model.fc.weight = coef_param
# net._model.fc.bias = bias_param

Parameter containing:
tensor([ 0.2977,  0.3007,  0.0680,  0.0970, -0.2821,  0.0656,  0.0192, -0.1751,
        -0.1088,  0.2491, -0.2712, -0.2794,  0.2055,  0.5799, -0.3717,  0.0535,
        -0.4479], device='cuda:0', requires_grad=True)

In [229]:
# Get test accuracies.
device = 'cuda'
criterion = utils.initialize(config['criterion']) 
get_all_test_stats(
    epoch=-1, test_loaders=test_loaders, max_test_examples=max_examples,
    config=config, net=net, criterion=criterion, device=device,
    loss_name_prefix='test_loss/', acc_name_prefix='test_acc/')


{'epoch': -1,
 'test_loss/source_val_living': 0.1388502937776071,
 'test_acc/source_val_living': 0.9647058823529412,
 'test_loss/target_val_living': 0.6023032312039975,
 'test_acc/target_val_living': 0.8194117647058824}

# Load using sklearn

In [202]:
# To check this, let's load features and test weights using sklearn.
features_path = '/u/scr/ananya/cifar_experiments/unlabeled_extrapolation/logs/linprobe_living17_resnet50/features_0'
features, labels, loader_names = pickle.load( open( features_path, "rb" ) )

In [203]:
new_coef, new_intercept = inv_normalize_weights(coef, intercept, features, 0)
clf = LogisticRegression(random_state=0, warm_start=True)
clf.coef_ = new_coef
clf.intercept_ = new_intercept
clf.classes_ = np.array(list(range(17)))

In [204]:
for idx in [1, 2]:
    preds = clf.predict(features[idx])
    print(np.mean(preds == labels[idx]))

0.9647058823529412
0.8194117647058824
